# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [65]:
# SETUP

from itertools import combinations, chain, count
from heapq import heappush, heappop
from collections import deque

# Number of couples
N = 3
people = [f'H{i+1}' for i in range(N)] + [f'W{i+1}' for i in range(N)]

In [67]:
# State represented as tuple of (left, island, right, boat_pos)
class State:
    def __init__(self, left, island, right, boat):
        self.left = frozenset(left)
        self.island = frozenset(island)
        self.right = frozenset(right)
        self.boat = boat

    def is_goal(self):
        return len(self.right) == 2 * N

    def __hash__(self):
        return hash((self.left, self.island, self.right, self.boat))

    def __eq__(self, other):
        return (self.left, self.island, self.right, self.boat) == (other.left, other.island, other.right, other.boat)

    def __str__(self):
        return f"L:{sorted(self.left)} | I:{sorted(self.island)} | R:{sorted(self.right)} | Boat:{self.boat}"

# Initial state
initial_state = State(set(people), set(), set(), 'L')

In [69]:
# Helper: Jealousy check
def is_valid(group):
    husbands = {p for p in group if p.startswith('H')}
    wives = {p for p in group if p.startswith('W')}
    
    for w in wives:
        h = 'H' + w[1:]
        if h not in group:
            for h_other in husbands:
                if h_other != h:
                    return False
    return True

def generate_children(state):
    locations = {
        'L': set(state.left),
        'I': set(state.island),
        'R': set(state.right)
    }

    transitions = {
        'L': 'I',
        'I': 'R',
        'R': 'I',
        'I-back': 'L'
    }

    results = []
    for direction in ['L', 'I', 'R']:
        if state.boat != direction:
            continue

        to = transitions[direction] if direction != 'I' else transitions['I']  # island -> right
        from_group = locations[direction]
        to_group = locations[to]

        for people_in_boat in chain(combinations(from_group, 1), combinations(from_group, 2)):
            new_from = from_group.difference(people_in_boat)
            new_to = to_group.union(people_in_boat)

            new_locations = dict(locations)
            new_locations[direction] = new_from
            new_locations[to] = new_to

            if all(is_valid(new_locations[loc]) for loc in ['L', 'I', 'R']) and is_valid(people_in_boat):
                new_boat = to
                new_state = State(new_locations['L'], new_locations['I'], new_locations['R'], new_boat)
                results.append(new_state)

        # Also allow island → left moves
        if direction == 'I':
            to = transitions['I-back']
            to_group = locations[to]
            for people_in_boat in chain(combinations(from_group, 1), combinations(from_group, 2)):
                new_from = from_group.difference(people_in_boat)
                new_to = to_group.union(people_in_boat)
                new_locations = dict(locations)
                new_locations['I'] = new_from
                new_locations[to] = new_to
                if all(is_valid(new_locations[loc]) for loc in ['L', 'I', 'R']) and is_valid(people_in_boat):
                    new_state = State(new_locations['L'], new_locations['I'], new_locations['R'], to)
                    results.append(new_state)

    return results

In [71]:
# DFS implementation
def dfs(state, visited=None):
    if visited is None:
        visited = set()
    if state.is_goal():
        return [state]
    visited.add(state)
    for child in generate_children(state):
        if child not in visited:
            result = dfs(child, visited)
            if result:
                return [state] + result
    return None

# Run DFS
dfs_result = dfs(initial_state)
dfs_result_str = [str(s) for s in dfs_result] if dfs_result else []

# Show results
dfs_result_str

["L:['H1', 'H2', 'H3', 'W1', 'W2', 'W3'] | I:[] | R:[] | Boat:L",
 "L:['H2', 'H3', 'W2', 'W3'] | I:['H1', 'W1'] | R:[] | Boat:I",
 "L:['H2', 'H3', 'W2', 'W3'] | I:[] | R:['H1', 'W1'] | Boat:R",
 "L:['H2', 'H3', 'W2', 'W3'] | I:['H1'] | R:['W1'] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W2', 'W3'] | I:[] | R:['W1'] | Boat:L",
 "L:['H1', 'H2', 'H3', 'W3'] | I:['W2'] | R:['W1'] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W3'] | I:[] | R:['W1', 'W2'] | Boat:R",
 "L:['H1', 'H2', 'H3', 'W3'] | I:['W1'] | R:['W2'] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W1', 'W3'] | I:[] | R:['W2'] | Boat:L",
 "L:['H1', 'H3', 'W1', 'W3'] | I:['H2'] | R:['W2'] | Boat:I",
 "L:['H1', 'H3', 'W1', 'W3'] | I:[] | R:['H2', 'W2'] | Boat:R",
 "L:['H1', 'H3', 'W1', 'W3'] | I:['H2', 'W2'] | R:[] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W1', 'W3'] | I:['W2'] | R:[] | Boat:L",
 "L:['H1', 'H2', 'H3', 'W1'] | I:['W2', 'W3'] | R:[] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W1'] | I:[] | R:['W2', 'W3'] | Boat:R",
 "L:['H1', 'H2', 'H3', 'W1'] | I:['W2'] | R:['

In [73]:
# Backtracking implementation
def backtrack(state, path=None, visited=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    if state.is_goal():
        return path + [state]
    visited.add(state)
    for child in generate_children(state):
        if child not in visited:
            result = backtrack(child, path + [state], visited)
            if result:
                return result
    return None

# Run Backtracking
backtrack_result = backtrack(initial_state)
backtrack_result_str = [str(s) for s in backtrack_result] if backtrack_result else []

# Show results
backtrack_result_str

["L:['H1', 'H2', 'H3', 'W1', 'W2', 'W3'] | I:[] | R:[] | Boat:L",
 "L:['H2', 'H3', 'W2', 'W3'] | I:['H1', 'W1'] | R:[] | Boat:I",
 "L:['H2', 'H3', 'W2', 'W3'] | I:[] | R:['H1', 'W1'] | Boat:R",
 "L:['H2', 'H3', 'W2', 'W3'] | I:['H1'] | R:['W1'] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W2', 'W3'] | I:[] | R:['W1'] | Boat:L",
 "L:['H1', 'H2', 'H3', 'W3'] | I:['W2'] | R:['W1'] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W3'] | I:[] | R:['W1', 'W2'] | Boat:R",
 "L:['H1', 'H2', 'H3', 'W3'] | I:['W1'] | R:['W2'] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W1', 'W3'] | I:[] | R:['W2'] | Boat:L",
 "L:['H1', 'H3', 'W1', 'W3'] | I:['H2'] | R:['W2'] | Boat:I",
 "L:['H1', 'H3', 'W1', 'W3'] | I:[] | R:['H2', 'W2'] | Boat:R",
 "L:['H1', 'H3', 'W1', 'W3'] | I:['H2', 'W2'] | R:[] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W1', 'W3'] | I:['W2'] | R:[] | Boat:L",
 "L:['H1', 'H2', 'H3', 'W1'] | I:['W2', 'W3'] | R:[] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W1'] | I:[] | R:['W2', 'W3'] | Boat:R",
 "L:['H1', 'H2', 'H3', 'W1'] | I:['W2'] | R:['

In [75]:
# Heuristic for A* search
def heuristic(state):
    return len(state.left)

# A* implementation
def a_star(start_state):
    frontier = []
    counter = count()  # Unique sequence count
    heappush(frontier, (heuristic(start_state), 0, next(counter), start_state, []))
    visited = set()
    while frontier:
        f, g, _, current, path = heappop(frontier)
        if current in visited:
            continue
        visited.add(current)
        if current.is_goal():
            return path + [current]
        for child in generate_children(current):
            if child not in visited:
                heappush(frontier, (g + 1 + heuristic(child), g + 1, next(counter), child, path + [current]))
    return None
    
# Run A*
astar_result = a_star(initial_state)
astar_result_str = [str(s) for s in astar_result] if astar_result else []

# Show results
astar_result_str

["L:['H1', 'H2', 'H3', 'W1', 'W2', 'W3'] | I:[] | R:[] | Boat:L",
 "L:['H2', 'H3', 'W2', 'W3'] | I:['H1', 'W1'] | R:[] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W2', 'W3'] | I:['W1'] | R:[] | Boat:L",
 "L:['H1', 'H2', 'H3'] | I:['W1', 'W2', 'W3'] | R:[] | Boat:I",
 "L:['H1', 'H2', 'H3', 'W1'] | I:['W2', 'W3'] | R:[] | Boat:L",
 "L:['H1', 'W1'] | I:['H2', 'H3', 'W2', 'W3'] | R:[] | Boat:I",
 "L:['H1', 'W1'] | I:['H2', 'W2'] | R:['H3', 'W3'] | Boat:R",
 "L:['H1', 'W1'] | I:['H2', 'H3', 'W2'] | R:['W3'] | Boat:I",
 "L:['H1', 'H3', 'W1'] | I:['H2', 'W2'] | R:['W3'] | Boat:L",
 "L:['W1'] | I:['H1', 'H2', 'H3', 'W2'] | R:['W3'] | Boat:I",
 "L:['W1'] | I:['H2', 'W2'] | R:['H1', 'H3', 'W3'] | Boat:R",
 "L:['W1'] | I:['H1', 'H2', 'W2'] | R:['H3', 'W3'] | Boat:I",
 "L:['H1', 'W1'] | I:['H2', 'W2'] | R:['H3', 'W3'] | Boat:L",
 "L:[] | I:['H1', 'H2', 'W1', 'W2'] | R:['H3', 'W3'] | Boat:I",
 "L:[] | I:['W1', 'W2'] | R:['H1', 'H2', 'H3', 'W3'] | Boat:R",
 "L:[] | I:['W1', 'W2', 'W3'] | R:['H1', 'H2', 'H3']